In [1]:
import pandas as pd
import tushare as ts
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import backtrader as bt

In [2]:
ts.set_token('5d3c09f16bc3ad409898ff6cfc10aadeeae28ba23aa240c047dd125c')
pro = ts.pro_api()

start_date = '20190101'
end_date = '20210531'
code = '600519.SH'

def get_data(code = code,start_date = start_date,end_date = end_date):
    df = pro.daily(ts_code = code,start_date = start_date,end_date = end_date)
    df = df.iloc[::-1] #序列倒转
    df.index = pd.to_datetime(df.trade_date) #index改为时间格式
    df['openinterest'] = 0 #设置Openinterest
    df = df[['open','high','low','close','vol','openinterest']]
    df.rename(columns={'vol': 'volume'}, inplace=True)
    return df

stock_df = get_data()
print(stock_df.head())
data = bt.feeds.PandasData(
    dataname=stock_df,
    fromdate=pd.to_datetime(start_date),
    todate=pd.to_datetime(end_date),
)

              open    high     low   close    volume  openinterest
trade_date                                                        
2019-01-02  609.98  612.00  595.01  598.98  62286.49             0
2019-01-03  599.97  601.66  585.80  590.00  30977.35             0
2019-01-04  587.28  607.67  582.02  602.00  37683.47             0
2019-01-07  608.00  612.00  602.22  605.49  34750.13             0
2019-01-08  605.50  612.00  600.28  604.79  28838.13             0


In [7]:
# Create a subclass of Strategy to define the indicators and logic

class Reverse(bt.Strategy):
    params = (
        ('exitbars', 5),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [9]:
cerebro = bt.Cerebro()  # create a "Cerebro" engine instance
cerebro.addstrategy(Reverse)  # Add the trading strategy

cerebro.adddata(data_bt)  # Add the data feed

startcash = 1000000  # initial cash
cerebro.broker.setcash(startcash)  # set the starting cash

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission - 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.001)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()  # run it all
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
#cerebro.plot()  # and plot it with a single command

Starting Portfolio Value: 1000000.00
2019-01-02, Close, 598.98
2019-01-02, BUY CREATE, 598.98
2019-01-03, BUY EXECUTED, Price: 599.97, Cost: 5999.70, Comm 6.00
2019-01-03, Close, 590.00
2019-01-04, Close, 602.00
2019-01-07, Close, 605.49
2019-01-08, Close, 604.79
2019-01-09, Close, 616.12
2019-01-10, Close, 618.77
2019-01-10, SELL CREATE, 618.77
2019-01-11, SELL EXECUTED, Price: 625.00, Cost: 5999.70, Comm 6.25
2019-01-11, OPERATION PROFIT, GROSS 250.30, NET 238.05
2019-01-11, Close, 635.88
2019-01-14, Close, 624.60
2019-01-15, Close, 659.98
2019-01-16, Close, 661.10
2019-01-17, Close, 659.25
2019-01-18, Close, 683.61
2019-01-21, Close, 686.98
2019-01-22, Close, 671.00
2019-01-23, Close, 665.66
2019-01-23, BUY CREATE, 665.66
2019-01-24, BUY EXECUTED, Price: 670.30, Cost: 6703.00, Comm 6.70
2019-01-24, Close, 672.50
2019-01-25, Close, 679.70
2019-01-28, Close, 677.00
2019-01-29, Close, 685.00
2019-01-30, Close, 676.00
2019-01-31, Close, 689.60
2019-01-31, SELL CREATE, 689.60
2019-02-01,